## Task  1: Loading Transactions from a CSV File

In [2]:
import csv
from datetime import datetime

In [4]:
def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""
    transactions = []
    try:
        # Open file with 'with' statement
        with open((filename), 'r') as file:
        # Use csv.DictReader
            reader = csv.DictReader(file)
        # For each row:
            for row in reader:
        #   Parse date with datetime.strptime
                row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
        #   Make amount negative for 'debit'
                if row['type'] == 'debit':
                    row['amount'] = -float(row['amount'])
                else:
                    row['amount'] = float(row['amount'])
        #   Create dictionary with all fields
                transaction = {
                    'transaction_id': row['transaction_id'],
                    'date': row['date'],
                    'customer_id': row['customer_id'],
                    'amount': row['amount'],
                    'type': row['type'],
                    'description': row['description'],
                }
        #   Add to transactions
                transactions.append(transaction)
    # Catch FileNotFoundError, ValueError
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except ValueError as e:
        print(f"Error: {e}. Please check the date format in the file.")
    
    return transactions

load_transactions()
    

Error: time data '2020a-10-26' does not match format '%Y-%m-%d'. Please check the date format in the file.


[]

### Additional tasks to be completed
1. Print the number of loaded transactions.
2. Skip rows with invalid amounts and print a warning.
3. Test with a CSV containing one bad date and verify it’s skipped.
4. Log errors to `errors.txt`.

In [5]:
# Print the number of loaded transactions
print (len(load_transactions()))



Error: time data '2020a-10-26' does not match format '%Y-%m-%d'. Please check the date format in the file.
0


In [6]:
# Skip rows with invalid amounts and print a warning.
def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""
    transactions = []
    try:
        # Open file with 'with' statement
        with open((filename), 'r') as file:
        # Use csv.DictReader
            reader = csv.DictReader(file)
        # For each row:
            for row in reader:
        #   Parse date with datetime.strptime
                # Skip rows with invalid dates
                try:
                    row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
                except ValueError:
                    print("Warning, invalid date detected, skipping row.")
                    # Log errors to errors.txt
                    with open("errors.txt", "w") as file:
                        file.write(f"Warning, invalid date detected in row: {row}\n")
                    continue
        #   Make amount negative for 'debit'
                try:
                    if row['type'] == 'debit':
                        row['amount'] = -float(row['amount'])
                    else:
                        row['amount'] = float(row['amount'])
                except ValueError:
                    print("Warning, invalid amount detected, skipping row.")
                    # Log errors to errors.txt
                    with open("errors.txt", "a") as file:
                        file.write(f"Warning, invalid amount detected in row: {row}\n")
                    continue
        #   Create dictionary with all fields
                transaction = {
                    'transaction_id': row['transaction_id'],
                    'date': row['date'],
                    'customer_id': row['customer_id'],
                    'amount': row['amount'],
                    'type': row['type'],
                    'description': row['description'],
                }
        #   Add to transactions
                transactions.append(transaction)
    # Catch FileNotFoundError, ValueError
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
        # Log errors to errors.txt
        with open("errors.txt", "a") as file:
            file.write(f"Error: The file {filename} was not found.")
    except ValueError as e:
        print(f"Error: {e}. Please check the date format in the file.")
        # Log errors to errors.txt
        with open("errors.txt", "a") as file:
            file.write(f"Error: {e}. Please check the date format in the file.")
    return transactions

load_transactions()

Warning, invalid date detected, skipping row.
Warning, invalid amount detected, skipping row.


[{'transaction_id': '2',
  'date': datetime.date(2020, 1, 8),
  'customer_id': '466',
  'amount': 1255.95,
  'type': 'credit',
  'description': 'Each left similar likely coach take.'},
 {'transaction_id': '4',
  'date': datetime.date(2020, 12, 2),
  'customer_id': '142',
  'amount': 2927.41,
  'type': 'credit',
  'description': 'Agree reveal buy black already.'},
 {'transaction_id': '5',
  'date': datetime.date(2020, 12, 2),
  'customer_id': '944',
  'amount': -4661.88,
  'type': 'debit',
  'description': 'Child relationship show college whom speech.'},
 {'transaction_id': '6',
  'date': datetime.date(2021, 4, 25),
  'customer_id': '900',
  'amount': 3649.68,
  'type': 'credit',
  'description': 'Century front item.'},
 {'transaction_id': '7',
  'date': datetime.date(2020, 6, 19),
  'customer_id': '458',
  'amount': 504.24,
  'type': 'credit',
  'description': 'Market cover loss hotel send.'},
 {'transaction_id': '8',
  'date': datetime.date(2022, 9, 8),
  'customer_id': '573',
  'amou

## Task 2: Adding and Viewing Transactions

In [9]:
def add_transaction(transactions):
    """Add a new transaction from user input."""
    # Prompt for date, customer_id, amount, type, description
    
    # Validate date
    while True:
        try:
            date = datetime.strptime(input("Enter date (YYYY-MM-DD): "), '%Y-%m-%d').date()
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")
            continue
    customer_id = input("Enter customer ID: ")
    
    # Validate amount
    while True:
        try:
            amount = float(input("Enter amount: "))
            break
        except ValueError:
            print("Invalid amount. Please enter a number.")
            continue
    # Validate type
    while True:
        type = input("Enter type (credit/debit/transfer): ").lower()
        if type == 'debit':
            amount = -abs(amount)  # Make amount negative for debit
            break
        elif type not in ['credit', 'transfer']:
            print("Invalid type. Please enter 'credit', 'debit', or 'transfer'.")
            continue
    description = input("Enter description: ")
  
    # Generate new transaction_id
    # Create dictionary and append
    pass

add_transaction(load_transactions())

Warning, invalid date detected, skipping row.
Warning, invalid amount detected, skipping row.
Invalid amount. Please enter a number.
Invalid amount. Please enter a number.
Invalid amount. Please enter a number.
Invalid type. Please enter 'credit', 'debit', or 'transfer'.
